# Programming GPT from scratch.

In [2]:
import re
import sys
import nltk
import numpy as np
from nltk.tokenize import RegexpTokenizer

### Milestone 1.


In [10]:
text = 'That U.S.A. poster-print costs $12.40...'

pattern = r'''(?x)            # verbose mode
    ([A-Z]\.)+              # abbreviations, e.g. U.S.A.
  | \w+(-\w+)*              # words with optional internal hyphens
  | \$?\d+(\.\d+)?%?        # currency and percentages
  | \.\.\.                    # ellipsis
  | [][.,;"'?():-_]           # these are separate tokens; includes ], [
'''

tokens = nltk.regexp_tokenize(text, pattern)
print(tokens)

[('', '', ''), ('A.', '', ''), ('', '-print', ''), ('', '', ''), ('', '', '.40'), ('', '', '')]


In [8]:
# class example
# tr -sc 'A-Za-z' '\n' <shakes.text

text = 'That U.S.A. poster-print costs $12.40...'

pattern = r'''(?x)            # verbose mode
    (?:[A-Z]\.)+              # abbreviations, e.g. U.S.A.
  | \w+(?:-\w+)*              # words with optional internal hyphens
  | \$?\d+(?:\.\d+)?%?        # currency and percentages
  | \.\.\.                    # ellipsis
  | [][.,;"'?():-_]           # these are separate tokens; includes ], [
'''

tokens = nltk.regexp_tokenize(text, pattern)
print(tokens)


['That', 'U.S.A.', 'poster-print', 'costs', '$12.40', '...']


In [ ]:
# Alternative using RegexpTokenizer from nltk

tokenizer = RegexpTokenizer(
    r'''(?x)               # verbose
        (?:[A-Z]\.)+       # U.S.A.
      | \w+(?:-\w+)*       # hyphenated words
      | \$?\d+(?:\.\d+)?%? # numbers, currency, percents
      | \.\.\.             # ellipsis
      | [][.,;"'?():-_]    # punctuation
    '''
)
print(tokenizer.tokenize(text))


['That', 'U.S.A.', 'poster-print', 'costs', '$12.40', '...']


### BPE token learner algorithm

In [11]:
def bpe(c, k): # function byte-paire encoding(string c, number of merges k) returns vocab V.
    """
    BPE token learner algorithm
    :param c: input string
    :param k: number of merges
    :return: vocabulary V
    """
    # Initialize vocabulary with characters in c
    vocab = {char: 1 for char in c}
    
    # Perform k merges
    for _ in range(k):
        # Find the most frequent pair of characters
        pairs = {}
        for i in range(len(c) - 1):
            pair = c[i:i + 2]
            if pair in pairs:
                pairs[pair] += 1
            else:
                pairs[pair] = 1
        
        # Find the most frequent pair
        most_frequent_pair = max(pairs, key=pairs.get)
        
        # Merge the most frequent pair in the vocabulary
        new_char = ''.join(most_frequent_pair)
        vocab[new_char] = pairs[most_frequent_pair]
        
        # Update the string c by replacing the most frequent pair with the new character
        c = c.replace(most_frequent_pair, new_char)
    
    return vocab    

In [15]:
tokens = bpe(text, 200)  # Example usage of BPE with 10 merges
print(tokens)  # Output the vocabulary after BPE

{'T': 1, 'h': 1, 'a': 1, 't': 1, ' ': 1, 'U': 1, '.': 1, 'S': 1, 'A': 1, 'p': 1, 'o': 1, 's': 1, 'e': 1, 'r': 1, '-': 1, 'i': 1, 'n': 1, 'c': 1, '$': 1, '1': 1, '2': 1, '4': 1, '0': 1, 't ': 2}


In [ ]:
def load_data():    
    #
    # Load data
    #

    file_path = "./../data/shakespeare.txt"
    with open(file_path) as file:
        corpus = file.read()

    #
    # Split into train and test
    #
    corpus = np.array(corpus.split("\n"))
    num_lines = len(corpus)

In [ ]:
# Split Shakespeare corpus into train/valid/test splits based on line sampling

import random

def split_shakespeare(
    input_path: str,
    test_frac: float = 0.01,
    seed: int = 42,
    out_prefix: str = "shakespeare"
):
    """
    Reads the input text file, shuffles its lines, and splits them into:
      - test set: 1% of lines
      - training set: remaining 99% of lines

    Saves the splits as:
      {out_prefix}.test.txt
      {out_prefix}.train.txt
    """
    # Read all lines
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Shuffle for randomness (reproducible via seed)
    random.seed(seed)
    random.shuffle(lines)

    n_total = len(lines)
    n_test = int(n_total * test_frac)

    test_lines = lines[:n_test]
    train_lines = lines[n_test:]

    # Write out the splits
    splits = {
        f"{out_prefix}.test.txt": test_lines,
        f"{out_prefix}.valid.txt": valid_lines,
        f"{out_prefix}.train.txt": train_lines,
    }
    for filename, subset in splits.items():
        with open(filename, 'w', encoding='utf-8') as out_f:
            out_f.writelines(subset)
        print(f"Wrote {len(subset)} lines to {filename}")

if __name__ == "__main__":
    split_shakespeare("shakespeare.txt")


### implimentation BPE

In [ ]:
# Byte-Pair Encoding with Detailed Debugging for Jupyter

import re
from collections import defaultdict

class BytePairEncodingDebug:
    def __init__(self, num_merges):
        """
        num_merges: number of BPE merge operations (k)
        """
        self.num_merges = num_merges
        # Initialize vocab to all printable ASCII (lowercase only) + newline
        self.vocab = [chr(i) for i in range(128) if chr(i).isprintable() and not chr(i).isupper()]
        self.vocab.append("\n")

    def normalize(self, word):
        # Lowercase normalization
        return word.lower()

    def train(self, corpus):
        """
        Train BPE on `corpus` (a single string).
        Prints detailed info at each merge step.
        Returns the final vocabulary list.
        """
        # 1) Split into "words" (space-separated) + count frequencies
        map_word_cnt = defaultdict(int)
        map_word_tokens = {}
        for raw_line in corpus.split("\n"):
            for w in raw_line.split(" "):
                word = self.normalize(w) + " "  # add end-of-word marker
                map_word_cnt[word] += 1
                if word not in map_word_tokens:
                    map_word_tokens[word] = list(word)

        # 2) Perform merges
        for merge_i in range(1, self.num_merges + 1):
            # 2.1) Count all adjacent pairs
            pair_counts = defaultdict(int)
            for word, tokens in map_word_tokens.items():
                freq = map_word_cnt[word]
                for j in range(len(tokens) - 1):
                    pair = (tokens[j], tokens[j+1])
                    pair_counts[pair] += freq

            # Debug print: top 5 most common pairs before merge
            top5 = sorted(pair_counts.items(), key=lambda x: -x[1])[:5]
            print(f"[Merge {merge_i}] Top-5 candidate pairs:", top5)

            # 2.2) Select the most frequent
            (tL, tR), count = max(pair_counts.items(), key=lambda x: x[1])
            new_token = tL + tR

            # Debug print: which pair is chosen
            print(f"[Merge {merge_i}] Chosen pair: ({tL!r}, {tR!r}) -> New token: {new_token!r} (count={count})")

            # 2.3) Add to vocab
            self.vocab.append(new_token)

            # 2.4) Replace in all words
            for word, tokens in map_word_tokens.items():
                new_tokens = []
                i = 0
                while i < len(tokens):
                    if i+1 < len(tokens) and tokens[i] == tL and tokens[i+1] == tR:
                        new_tokens.append(new_token)
                        i += 2
                    else:
                        new_tokens.append(tokens[i])
                        i += 1
                map_word_tokens[word] = new_tokens

            print(f"[Merge {merge_i}] Vocab size now: {len(self.vocab)}\n" + "-"*50)

        # 3) Sort vocab longest-first for greedy tokenization
        self.vocab.sort(key=len, reverse=True)
        return self.vocab

    def segment(self, text):
        """
        Tokenize `text` using the learned BPE vocab.
        """
        tokens_out = []
        for line in text.split("\n"):
            for w in line.split(" "):
                word = self.normalize(w) + " "
                tokens_out.extend(self._tokenize_word(word))
        return tokens_out[:-1]  # drop final dummy space

    def _tokenize_word(self, word):
        """
        Greedy longest-match tokenization of a single word+space.
        """
        i = 0
        out = []
        while i < len(word):
            for t in self.vocab:
                if word.startswith(t, i):
                    out.append(t)
                    i += len(t)
                    break
        return out





In [ ]:
# 1) Load your corpus:
with open("data/shakespeare.txt","r") as f:
    text = f.read()
#
# 2) Instantiate with desired merges (e.g., 10):
bpe = BytePairEncodingDebug(num_merges=10)
#
# 3) Train and watch the debug prints for each merge:
vocab = bpe.train(text)
#
# 4) Inspect the final vocab:
print("Final vocab:", vocab)
#
print('\n\n')
# 5) Segment some sample sentences:
print(bpe.segment(f"To be or not to be"))

[Merge 1] Top-5 candidate pairs: [(('e', ' '), 129069), (('t', 'h'), 113368), (('h', 'e'), 83033), ((',', ' '), 82969), (('t', ' '), 77622)]
[Merge 1] Chosen pair: ('e', ' ') -> New token: 'e ' (count=129069)
[Merge 1] Vocab size now: 71
--------------------------------------------------
[Merge 2] Top-5 candidate pairs: [(('t', 'h'), 113368), ((',', ' '), 82969), (('t', ' '), 77622), (('.', ' '), 76602), (('s', ' '), 75157)]
[Merge 2] Chosen pair: ('t', 'h') -> New token: 'th' (count=113368)
[Merge 2] Vocab size now: 72
--------------------------------------------------
[Merge 3] Top-5 candidate pairs: [((',', ' '), 82969), (('t', ' '), 77622), (('.', ' '), 76602), (('s', ' '), 75157), (('d', ' '), 67811)]
[Merge 3] Chosen pair: (',', ' ') -> New token: ', ' (count=82969)
[Merge 3] Vocab size now: 73
--------------------------------------------------
[Merge 4] Top-5 candidate pairs: [(('t', ' '), 77622), (('.', ' '), 76602), (('s', ' '), 75157), (('d', ' '), 67811), (('a', 'n'), 61157)